In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as matplot
import itertools
import math

from tqdm import tqdm_notebook, tqdm
from os import listdir, mkdir
from os.path import join, isdir
from ipywidgets import *
from multiprocessing import Pool, Lock

import nothing

In [ ]:
import sys
module_path = os.path.abspath(os.path.join('../python/python_src_preprocessing/src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import utils.writer.density as density

%matplotlib notebook

In [2]:
LOGGING = False

In [3]:
def log(*args):
    if LOGGING:
        print(*args)

def load_experiment(folder, key):
    files = list(filter(lambda file: key in file, listdir(folder)))
    
    if len(files) == 0:
        return None
    
    data = pd.read_csv(
        join(folder, files[0]), 
        sep=' ', 
        names=['p-id', 'timestep', 'x', 'y', 'e'], 
        index_col=False, 
        header=None, 
        skiprows=0)
        
    data['x'] = data['x'] / 100
    data['y'] = data['y'] / 100
    
    print('loaded', files[0])
    
    return files[0], data
    
def load_multiple(path, folders, key):
    frames = []
    for folder in folders:
        frames.append(load_experiment(join(path, folder), key))
            
    return frames

In [ ]:
def plot_experiment(frame, title):
    fig, ax = matplot.subplots()
    ax.set_title(title)
    
    # map p-ids to targets
    for pId, group in frame.groupby('p-id'):
        ax.plot(group['x'], group['y'])
    
    ax.grid()

In [ ]:
def extract_observation_area(frame, area):
    is_x = (frame['x'] >= area[0]) & (frame['x'] <= (area[0] + area[2]))
    is_y = (frame['y'] >= area[1]) & (frame['y'] <= (area[1] + area[3]))
    return frame[is_x & is_y]

In [ ]:
def get_target_percentiles(pedestrians, ped2target, targets):
    if len(pedestrians)== 0:
        return None
    
    ids = list(pedestrians['p-id'])
    total = len(ids)
    
    filtered_dict = {k:v for k,v in ped2target.items() if k in ids}
    used_targets = filtered_dict.values()
    
    percentiles = {k:(len(list(v)) / total) for k, v in itertools.groupby(sorted(used_targets))}
    
    for k in targets:
        if k not in used_targets:
            percentiles[k] = 0.0
    
    return percentiles

In [ ]:
def get_gaussian_grid(gauss_bound, resolution, sigma, ped_pos, radius):
    x = np.arange(-gauss_bound, gauss_bound + resolution, resolution) # gives gauss_bound_start:resolution:gauss_bound_stop
    
    xx, yy = np.meshgrid(x, x, sparse=False) # Make all grid points (based on resolution) in [gauss_bound_start, gauss_bound_stop] (2-dim-array)
        
    grid = np.sqrt(np.square(xx-ped_pos[0]) + np.square(yy-ped_pos[1])) # distance to the origin of the observation area
    
    gauss = np.vectorize(gaussian_pdf)

    gauss_grid = gauss(sigma, grid, radius)
    
    return gauss_grid


def gaussian_pdf(sigma, x, radius ):
    zaehler = ((radius*2)**2) * np.sqrt(3)  # S_p
    nenner = 2 * 2 * np.pi * (sigma**2)

    normalization_factor = zaehler/nenner
    individual_density = normalization_factor * np.exp(-x / (2 * np.square(sigma)))

    return individual_density


def add_pedestrian_to_field(ped, matrix, area, resolution, gauss_density_bound, sigma, ped_radius, general_density_matrix):
    if general_density_matrix is None: # not None
        bool_individual_position = True
    else:
        bool_individual_position = False

    # calculate the density for one ped and add to matrix
    size = int(gauss_density_bound * 2 / resolution + 1)
    radius = int(size / 2) # equal to gauss_density_bound/resolution
    log('size', size, 'radius', radius)
    
    origin_x = area[0]
    origin_y = area[1]
    log('origin x', origin_x, 'origin y', origin_y)
    
    width = int(area[2] / resolution)
    height = int(area[3] / resolution)
    log('width', width, 'height', height)
    
    
    # necessary to map to center of the grid instead of the edge!
    offset = resolution / 2
    log('offset', offset)
    
    
    # find grid cell of pedestrian in observation area
    diff_x = int(np.round((ped['x'] - origin_x - offset) / resolution, 0))
    diff_y = int(np.round((ped['y'] - origin_y - offset) / resolution, 0))

    log('diff x', diff_x, 'diff y', diff_y)
    
    # area in which the pedestrian has an influence on the pedestrian density
    left_bound = int(max(0, diff_x - radius))
    right_bound = int(min(diff_x + radius, width - 1))
    upper_bound = int(min(diff_y + radius, height - 1))
    lower_bound = int(max(0, diff_y - radius))
    
    log('left', left_bound, 'right', right_bound)
    log('upper', upper_bound, 'lower', lower_bound)

    ## create density_field
    # position of pedestrian relative to center of grid cell
    grid_points_x = np.arange(origin_x+offset, origin_x + area[2] + resolution, resolution) # gives gauss_bound_start:resolution:gauss_bound_stop
    grid_points_y = np.arange(origin_y+offset, origin_y + area[3] + resolution, resolution)

    # center of cell in which the pedestrian is located
    grid_point_x = grid_points_x[diff_x]
    grid_point_y = grid_points_y[diff_y]

    ped_pos_rel_x = ped['x'] - grid_point_x
    ped_pos_rel_y = ped['y'] - grid_point_y

    log(ped_pos_rel_x, ped_pos_rel_y)
    
    # Position of pedestrian relative to cell center
    ped_pos_rel = np.array([ped_pos_rel_x, ped_pos_rel_y])

    log(bool_individual_position)
    
    if bool_individual_position:
        density_field = get_gaussian_grid(gauss_density_bound, resolution, sigma, ped_pos_rel, ped_radius)
    else:
        density_field = general_density_matrix
    
    '''
    fig, ax = matplot.subplots()
    img = ax.imshow(density_field, interpolation='Nearest')
    fig.colorbar(img)
    '''
    
    # cutout of density field that is within the camera cutout
    left_bound_field = max(0, radius - diff_x)
    right_bound_field = left_bound_field + right_bound - left_bound

    lower_bound_field = max(0, radius - diff_y)
    upper_bound_field = lower_bound_field + upper_bound - lower_bound

    matrix[lower_bound:upper_bound+1, left_bound:right_bound+1] = \
        matrix[lower_bound:upper_bound+1, left_bound:right_bound+1] + density_field[lower_bound_field:upper_bound_field+1, left_bound_field:right_bound_field+1]

    return matrix
    

def get_density_field(pedestrians, context):
    density_field = context.get('density_field', None)
    
    if density_field is not None:
        print(density_field)
    
    size = context.get('size', (0, 0))
    
    density_approx = np.zeros(size)
    ped_list = np.zeros([len(pedestrians),2])
    
    idx = 0
    for _, pedestrian in pedestrians.iterrows():
        density_approx = add_pedestrian_to_field(
            pedestrian, 
            density_approx, 
            context.get('area', None), 
            context.get('resolution', 1), 
            context.get('gauss_bounds', 1), 
            context.get('sigma', 0.7),
            context.get('pedestrian_radius', 0.195),
            density_field)

        ped_list[idx,:] = [pedestrian['x'], pedestrian['y']]
        idx += 1
        

    return density_approx

In [ ]:
def process_percentiles(pedestrians, percentiles, density, context):
    data = context.get('percentiles', None)
    
    # create dataframe if not yet available
    if data is None:
        data = pd.DataFrame(columns=context.get('targets'))
    
    context['percentiles'] = data.append(percentiles, ignore_index=True)
    
    return context

def process_pedestrians(pedestrians, percentiles, density, context):
    data = context.get('pedestrians', None)
    
    # create dataframe if not yet available
    if data is None:
        data = []
    
    data.append(pedestrians)
    
    context['pedestrians'] = data
    
    return context
    
    
def process_peds_per_step(pedestrians, percentiles, density, context):
    data = context.get('pedestrians', None)
    
    # create dataframe if not yet available
    if data is None:
        data = pd.DataFrame(columns=['#peds'])
        
    context['pedestrians'] = data.append({'#peds': len(peds) }, ignore_index=True)
    
    return context
    
    
def process_densities(pedestrians, percentiles, density, context):
    data = context.get('densities', None)
    
    # create dataframe if not yet available
    if data is None:
        data = []
        context['densities'] = data
    
    distribution = [percentiles[key] for key in sorted(percentiles.keys())]
    data.append(np.concatenate((density.flatten(), distribution), axis=None))
    
    return context

'''
def process_timestep(timestep, trajectories, context):
    pedestrians = extract_observation_area(trajectories, area)
        
    # skip if empty area
    if len(pedestrians) == 0:
        return

    percentiles = get_target_percentiles(pedestrians, pId2Target, context.get('targets', ['A', 'B']))

    if percentiles['A'] == 0.5 and percentiles['B'] == 0.5:
        flag = not(flag)
        if flag:
            return


    density = get_density_field(pedestrians, context)

    # execute processors
    for processor in processors:
        context = processor(pedestrians, percentiles, density, context)
        
def process_experiment(experiment, context):
    name = context.get('name', '')
    
    processors = context.get('processors', None)
    if processors is None or type(processors) != list:
        processors = []
    
    pId2Target = dict.fromkeys(list(frame['p-id'].unique()))
    
    # map p-ids to targets
    for pId, group in frame.groupby('p-id'):
        pId2Target[pId] = ('B' if group['x'].iloc[0] < 0 else 'A')
    
    if not context.get('exact', False):
        context['density_field'] = get_gaussian_grid(
            context.get('gauss_bounds', 0),
            context.get('resolution', 1),
            context.get('sigma', 1),
            [0, 0])
    
    area = context.get('area')
    resolution = context.get('resolution')
    
    size = (int(area[3]/ resolution), int(area[2] / resolution))
    context['size'] = size
    context['pId2Target'] = pId2Target
    
    flag = True
    
    for timestep, trajectories in tqdm_notebook(frame.groupby('timestep'), desc=name, unit=' timestep'):
        # extract #-of peds in observation area
        pedestrians = extract_observation_area(trajectories, area)
        
        # skip if empty area
        if len(pedestrians) == 0:
            continue
            
        percentiles = get_target_percentiles(pedestrians, pId2Target, context.get('targets', ['A', 'B']))
        
        if percentiles['A'] == 0.5 and percentiles['B'] == 0.5:
            flag = not(flag)
            if flag:
                continue
            
        
        density = get_density_field(pedestrians, context)
        
        # execute processors
        for processor in processors:
            context = processor(pedestrians, percentiles, density, context)
        
        
    return context
    
'''

In [ ]:
def process_experiment(experiment, context):
    name = context.get('name', '')
    
    processors = context.get('processors', None)
    if processors is None or type(processors) != list:
        processors = []
    
    pId2Target = dict.fromkeys(list(frame['p-id'].unique()))
    
    # map p-ids to targets
    for pId, group in frame.groupby('p-id'):
        pId2Target[pId] = ('B' if group['x'].iloc[0] < 0 else 'A')
    
    if not context.get('exact', False):
        context['density_field'] = get_gaussian_grid(
            context.get('gauss_bounds', 0),
            context.get('resolution', 1),
            context.get('sigma', 1),
            [0, 0])
    
    area = context.get('area')
    resolution = context.get('resolution')
    
    size = (int(area[3]/ resolution), int(area[2] / resolution))
    context['size'] = size
    context['pId2Target'] = pId2Target
    
    flag = True
    
    timesteps = list(experiment.groupby('timestep'))
    total = len(timesteps)
    timesteps = list(map(lambda a: (*a, context), timesteps))
    
    lock = Lock()
    
    with Pool(processes=4) as p:
        with tqdm_notebook(total=total) as pbar:
            for i, result in enumerate(p.imap_unordered(nothing.doNothing, timesteps)):
                pbar.update()
                
                pedestrians, percentiles, density = result
                
                if pedestrians is None:
                    continue
                
                
                if percentiles['A'] == 0.5 and percentiles['B'] == 0.5:
                    lock.acquire()
                    flag = not(flag)
                    lock.release()
                    if flag:
                        continue
                    
                # execute processors
                lock.acquire()
                for processor in processors:
                    context = processor(pedestrians, percentiles, density, context)
                lock.release()
                    
                
    return context

In [4]:
input_path = 'juelich/KO/'
output_path = 'juelich/output_01/'
experiments = [
    'ko-240-050-240',
    'ko-240-060-240',
    'ko-240-080-240',
    'ko-240-100-240',
    'ko-240-120-240',
    'ko-240-150-240',
    'ko-240-240-240',
    'ko-300-050-300',
    'ko-300-080-300',
    'ko-300-120-300',
    'ko-300-150-300'
]

# frames = load_experiment(join(input_path, experiments[0]), 'combined')

# frames = load_multiple(input_path, experiments, 'combined')

if not isdir(output_path):
    mkdir(output_path)

In [5]:
import nothing

results = []
for experiment in tqdm_notebook(experiments, desc='process files'):
    # plot_experiment(frame, experiment)
    filename, frame = load_experiment(join(input_path, experiment), 'combined')
    name = filename[:filename.index('.')]
    context = dict({
        'name': name,
        'targets': ['A', 'B', 'C'],
        'area':  [-2.4, 3, 2.4, 1],
        'exact': True,
        'sigma': 0.7,
        'resolution': 0.1,
        'pedestrian_radius': 0.195,
        'gauss_bounds': 2,
        'processors': [
            nothing.process_densities
        ]
    })
    context = nothing.process_experiment(frame, context)
    results.append(context)
    
    densities = context.get('densities')
    with open(join(output_path, name + '.csv'), mode='w') as file:
        for density in densities:
            file.write(';'.join(map(lambda r: "{:.10f}".format(r), density)) + '\n')
        
        file.flush()
    
    
'''
for result in tqdm_notebook(results, desc='saving density files'):
    name = result.get('name')
    densities = result.get('densities')
    
    with open(join(output_path, name + '.csv'), mode='w') as file:
        for density in densities:
            file.write(';'.join(map(lambda r: "{:.10f}".format(r), density)) + '\n')
        
        file.flush()
'''

loaded ko-240-050-240_combined_MB.txt


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 168.99it/s]


loaded ko-240-060-240_combined_MB.txt


100%|███████████████████████████████████████████████████████████████████████████████| 942/942 [00:06<00:00, 154.33it/s]


loaded ko-240-080-240_combined_MB.txt


...

loaded ko-300-120-300_combined_MB.txt


100%|██████████████████████████████████████████████████████████████████████████████| 1269/1269 [00:12<00:00, 97.69it/s]


loaded ko-300-150-300_combined_MB.txt


100%|█████████████████████████████████████████████████████████████████████████████| 1240/1240 [00:13<00:00, 122.98it/s]


'\nfor result in tqdm_notebook(results, desc=\'saving density files\'):\n    name = result.get(\'name\')\n    densities = result.get(\'densities\')\n    \n    with open(join(output_path, name + \'.csv\'), mode=\'w\') as file:\n        for density in densities:\n            file.write(\';\'.join(map(lambda r: "{:.10f}".format(r), density)) + \'\n\')\n        \n        file.flush()\n'

In [ ]:
def extrac_xy(pedestrians, x0, y0, resolution):
    x = []
    y = []
    
    for _, ped in pedestrians.iterrows():
        x.append(int(abs(ped['x'] - x0) * ( 1 / resolution)))
        y.append(int(abs(ped['y'] - y0) * ( 1 / resolution)))
    
    return x, y
        

for result in tqdm_notebook(results, desc='process results'):
    name = result.get('name')
    densities = result.get('densities')
    area = context.get('area')
    peds = context.get('pedestrians')
    size = result.get('size')
    res = result.get('resolution')
    
    start = 12
    
    first = densities[start][:-3].reshape(size)
    #first = first / np.max(first)
    
    fig, ax = matplot.subplots()
    
    #img = ax.imshow(first, interpolation='Nearest')
    cbar = None #fig.colorbar(img)
    
    #x, y = extrac_xy(peds[0], area[0], area[1], res)

    #pl = ax.plot(x, y, '.r', markersize=10)
    
    ax.set_xticks(np.linspace(0, len(first[0]) - 1, 13))
    ax.set_xticklabels([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4])
    
    ax.set_yticks(np.linspace(0, len(first) - 1, 6))
    ax.set_yticklabels([0, 0.2, 0.4, 0.6, 0.8, 1.0])
    
    
    
    def update(frame):
        global cbar
        f = densities[frame][:-3].reshape(size)
        
        ax.clear()
        if cbar is not None:
            cbar.remove()
        
        img = ax.imshow(f, interpolation='Nearest')
        cbar = fig.colorbar(img)

        x, y = extrac_xy(peds[frame], area[0], area[1], res)
        print(list(zip(x, y)))
        ax.plot(x, y, '.r', markersize=10)

        ax.set_xticks(np.linspace(0, len(first[0]) - 1, 13))
        ax.set_xticklabels([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4])

        ax.set_yticks(np.linspace(0, len(first) - 1, 6))
        ax.set_yticklabels([0, 0.2, 0.4, 0.6, 0.8, 1.0])
        

    interact(update, frame=widgets.IntSlider(min=0, max=len(densities)-1, step=1, value=start));
    
    break
    
    

In [ ]:
percentiles = pd.concat([result['percentiles'] for result in results])
display(percentiles.groupby(result.get('targets'), as_index=False).size().to_frame('frames').reset_index())

In [ ]:
def gaussian(R, x, z):
    a = 1 / (2 * np.pi * (R**2))
    b = -1 / (2 * (R**2))
    
    return a * math.exp(b * (np.linalg.norm(np.array(x) - np.array(z))**2))

def density(pedestrians, z, R, g):
    S_p = g**2 * math.sqrt(3) / 2
    D_p = S_p * np.sum(list(map(lambda x: gaussian(R, x, z) ,pedestrians)))
    
    return D_p

def generate(points):
    x = np.arange(-1, 1 + 0.1, 0.1)
    y = np.arange(-1, 1 + 0.1, 0.1)

    xx, yy = np.meshgrid(x, y, sparse=False)

    matrix = np.zeros((len(y), len(x)))
    if len(points) > 0:
        for i in range(0, len(y)):
            for j in range(0, len(x)):
                matrix[i, j] = density(points, [xx[i, j], yy[i, j]], 0.7, 0.195)
            
    return matrix

In [ ]:
A = generate([[0.45, 0]])
B = generate([[-0.8, 0]])

C = np.zeros((10, 24))

'''
C[:, 6:22] = C[:, 6:22] + A[:, 6:22]
C[:, 0:10] = C[:, 0:10] + B[:, 0:10]
'''

fig, ax = matplot.subplots()
img = ax.imshow(A)
fig.colorbar(img)

fig, ax = matplot.subplots()
img = ax.imshow(B)
fig.colorbar(img)

'''
fig, ax = matplot.subplots()
img = ax.imshow(A, vmin=0, vmax=0.011)
fig.colorbar(img)
'''

fig, ax = matplot.subplots()
img = ax.imshow(C)
fig.colorbar(img)
        


In [ ]:
# A = generate([[-0.8, 0]])
# B = generate([[0.8, 0]])
y = int(1 / 0.1)
x = int(2.4 / 0.1)

matrix = np.zeros((y, x))
add_pedestrian_to_field({'x': -0.75, 'y': 3.5}, matrix, [-2.4, 3, 2.4, 1], 0.1, .5, 0.7, 0.195, None)
add_pedestrian_to_field({'x': -2, 'y': 3.5}, matrix, [-2.4, 3, 2.4, 1], 0.1, .5, 0.7, 0.195, None)

fig, ax = matplot.subplots()
img = ax.imshow(matrix)
fig.colorbar(img)

In [ ]:
A = get_gaussian_grid(1, .1, 0.7, [-0, 0.04999999999999982], 0.195)

fig, ax = matplot.subplots()
img = ax.imshow(A)
fig.colorbar(img)